In [4]:
%reload_ext autoreload
%autoreload 2
import sys; sys.path.append('./src')
import urllib.request as urllib
from bs4 import BeautifulSoup
import re
from contextlib import closing
import sqlite3
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from DB import *

In [6]:
low_category_db='../data/database/1122_low_rate_bakawall_nolimit_prodcuts.db'
high_category_db='../data/database/1122_high_rate_bakawall_nolimit_prodcuts.db'
Hconn2,Hc2=get_db_conn(high_category_db)
Lconn2,Lc2=get_db_conn(low_category_db)

# for i in range(1,11):
#     Hc2.execute(f'alter table products add column category{i}')

# for i in range(1,11):
#     Lc2.execute(f'alter table products add column category{i}')

In [7]:
def get_each_product_info(url):
    '''
    レビュー詳細URLから必要な情報を手に入れるコード
    SQLに紐づけて必要な部分を後から埋めるでもいいかもしれない。
    categoryの数が変動するので注意
    '''
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    text=soup.select('.review-text')[0].text
    
    product_url='https://www.amazon.co.jp'+soup.select('.a-text-ellipsis > a')[0].get('href')
    category=get_category(product_url)
    
    info_list=[]
    info_list.extend([len(text),text])
    info_list.extend(category)
    info_list.extend(['']*(11-len(info_list)))
    
    return(tuple(info_list))

In [14]:
def get_category(product_url):
    browser.get(product_url)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    try:
        a=soup.select('.a-unordered-list')[0].text
    except:
        browser.get(product_url)
        soup = BeautifulSoup(browser.page_source, 'lxml')
    
    try:
        a=soup.select('.a-unordered-list')[0].text
    except:
        print('no_product')
        return(['no_product'])  
    category=soup.select('.a-unordered-list')[0].text
    category=re.sub('›','',category)
    category=re.sub('\n','',category)
    category=category.strip()
    category=category.split()
    print(category)
    return(category)  

    

In [16]:
Lc2.execute('''select distinct name,id,rate,url from products''')
for i,row in enumerate(Lc2):
    if i <=0:
        continue
    pd_url=row[3]
    print(i,pd_url)
    row+=get_each_product_info(pd_url)
    insert_sql = 'insert into products (review,text,category1,category2, category3, category4, category5, category6, category7, category8, category9) values (?,?,?,?,?,?,?,?,?,?,?)'
    Lc2.execute(insert_sql, row)
    Lconn2.commit()

1 https://www.amazon.co.jp/gp/customer-reviews/R2W1GSMKFX2GO2?ref=pf_vv_at_pdctrvw_srp
['本', 'スポーツ・アウトドア', '登山・ハイキング']


ProgrammingError: Incorrect number of bindings supplied. The current statement uses 11, and there are 15 supplied.

In [ ]:
Lc.execute('''select distinct name,id,rate,url from products''')
for row in Lc:
    each_id=row[0]
    username=row[1]
    baka_star=row[2]
    pd_url=row[3]
    i+=1
        print(i,pd_url)
        row+=get_each_product_info(pd_url)
        insert_sql = 'insert into products (review,text,category1, category2, category3, category4, category5, category6, category7, category8, category9) values (?,?,?,?,?,?,?,?,?,?,?)'
        Lc2.execute(insert_sql, row)
        Lconn2.commit()